#### Code with new equations of motion

 - still need power variable

In [ ]:
g = 3.711  # m/s^2, gravity on Mars
m = 1000  #kg , Mass of Lander

def simulate(X0, V0, land, landing_site, 
             fuel=200, dt=0.1, Nstep=1000, 
             autopilot=None, print_interval=100, parameters=None):
    
    n = len(X0)       # number of degrees of freedom (2 here)
    X = X0.copy()     # current position
    V = V0.copy()     # current velocity
    Xs = np.zeros((Nstep, n)) # position history (trajectory) 
    Vs = np.zeros((Nstep, n)) # velocity history
    thrust = np.zeros((Nstep, n)) # thrust history
    success = False
    fuel_warning_printed = False
    rotate = 0           # degrees, initial angle
    power = 0            # m/s^2, initial thrust power    
    
    for i in range(Nstep):
        Xs[i, :] = X     # Store positions
        Vs[i, :] = V     # Store velocities
        
        if autopilot is not None:
            # call user-supplied function to set `rotate` and `power`
            rotate, power = autopilot(i, X, V, fuel, rotate, power, parameters)
            assert abs(rotate) <= 90
            assert 0 <= power <= 4000
        
            rotate_rad = rotate * np.pi / 180.0 # degrees to radians
            ### CHANGE HERE ??
            thrust[i, :] = power * np.array([np.sin(rotate_rad), 
                                             np.cos(rotate_rad)])
            if fuel <= 0: 
                if not fuel_warning_printed:
                    print("Fuel empty! Setting thrust to zero")
                    fuel_warning_printed = True
                thrust[i, :] = 0
            else:
                fuel -= power*dt #change with thrust + tsfc 
        
        ### AND CHANGE HERE
        A = np.array([0, -g]) + thrust[i, :]/ m # acceleration  
        # g and not W cause W/m = g
        
        #OLD A = np.array([0, -g]) + thrust[i, :] # acceleration 
        V += A * dt                          # update velocities
        X += V * dt                          # update positions
        
        if i % print_interval == 0: 
            print(f"i={i:03d} X=[{X[0]:8.3f} {X[1]:8.3f}] V=[{V[0]:8.3f} {V[1]:8.3f}]"
                  f" thrust=[{thrust[i, 0]:8.3f} {thrust[i, 1]:8.3f}] fuel={fuel:8.3f}") #format of the lines (python f')

In [ ]:
# ALSO CHANGE POWER HERE !!

def proportional_autopilot(i, X, V, fuel, rotate, power,parameters):
    K_h,K_p = parameters
    c = 10.0 # target landing speed, m/s
    #K_h = 0.01                            #K_H HERE
    #K_p = 0.2                              #K_P HERE
    h = height(land, X)
    e = - (c + K_h*h + V[1])          
    Pout = K_p*e                     
    power = min(max(Pout, 0.0), 4000)
    if i % 100 == 0:
        print(f'e={e:8.3f} Pout={Pout:8.3f} power={power:8.3f}')     
    return (rotate, power)